In [1]:
import pandas as pd
import numpy as np

# Compile temporal independent variables
- <b>Goal</b>: Compile all temporal independent variables for further regression analysis. 
- This involves matching the country names (using a crosswalk) and years. Assumptions on how missing data is treated can be found in either the respective processing files (usually to fill the missing years), or below sections (to fill the missing countries)
- The groups of variables include:
    1. Climate opinion survey -> no data extrapolation 
    2. World development index: GDP, share of coal or gas in electricity, or share in GDP -> no data extrapolation 
    3. Energy statistics review: coal and gas reserves and production
    4. Fuel trade diversity and gravity 
    5. POLCON dataset: existance of judiciary, legislative etc. chambers and consistency
    6. Power sector reform: e.g. liberalization, consumer choice
    7. State capacity 
    8. Worldwide governance 
- To do list:
    9. Available LNG terminal 
    10. World value survey

In [2]:
# crosswalk - standardize with ISO3 at the end
crosswalk = pd.read_excel("../_data_process/country_names.xlsx")
crosswalk

,Country Code,Region,IncomeGroup,TableName,GEM_Name,Hanson_Name,BP_Name,Meta_Name,WEF_Name,Henisz_Name,Reform_Name,MESSAGE_Name
0,ABW,Latin America & Caribbean,High income,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,Aruba,NaN
1,AFG,South Asia,Low income,Afghanistan,NaN,Afghanistan,Afghanistan,NaN,NaN,Afghanistan,Afghanistan,Afghanistan
2,AGO,Sub-Saharan Africa,Lower middle income,Angola,NaN,Angola,NaN,Angola,Angola,Angola,Angola,Angola
3,ALB,Europe & Central Asia,Upper middle income,Albania,NaN,Albania,Albania,Albania,Albania,Albania,Albania,Albania
4,AND,Europe & Central Asia,High income,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,Andorra,Andorra
...,...,...,...,...,...,...,...,...,...,...,...,...
211,XKX,Europe & Central Asia,Upper middle income,Kosovo,Kosovo,Kosovo,NaN,Kosovo,NaN,Kosovo,Kosovo,NaN
212,YEM,Middle East & North Africa,Low income,"Yemen, Rep.",Yemen,Yemen,NaN,Yemen,Yemen,NaN,"Yemen, Rep.",Yemen
213,ZAF,Sub-Saharan Africa,Upper middle income,South Africa,South Africa,South Africa,South Africa,South Africa,South Africa,South Africa,South Africa,South Africa
214,ZMB,Sub-Saharan Africa,Low income,Zambia,Zambia,Zambia,Zambia,Zambia,Zambia,Zambia,Zambia,Zambia


### Climate opinion survey -  $\color{orange}{\text{social}}$

In [3]:
climate_survey = pd.read_csv("../_data_process/_all_temporal_climate_opinion.csv")
climate_survey

,country,year,Survey_Worry_%,Survey_Belief_%,Survey_FutureGen_%,Survey_Personal_%,Survey_FossilMore_%,Survey_FossilLess_%
0,Argentina,2021,88.000000,56.000000,68.000000,38.000000,21.000000,58.000000
1,Australia,2021,72.000000,50.000000,61.000000,19.000000,11.000000,71.000000
2,Brazil,2021,86.000000,53.000000,74.000000,56.000000,27.000000,54.000000
3,Canada,2021,78.000000,50.000000,64.000000,18.000000,12.000000,69.000000
4,Colombia,2021,90.000000,54.000000,77.000000,58.000000,26.000000,53.000000
...,...,...,...,...,...,...,...,...
136,Uruguay,2022,85.160842,56.147711,67.296866,32.039047,19.608326,61.067272
137,Uzbekistan,2022,78.558349,36.898455,36.332476,13.533452,24.500367,41.988850
138,Vietnam,2022,85.194041,27.349717,71.302560,51.805902,40.738441,35.108095
139,Yemen,2022,31.703961,20.528497,23.389677,11.408358,38.808648,18.752107


In [4]:
print(len(climate_survey["country"].unique()))
print(climate_survey["year"].min())
print(climate_survey["year"].max())
print(round(climate_survey["Survey_Belief_%"].mean(),2))
print(round(climate_survey["Survey_Belief_%"].std(),2))
print(round(climate_survey["Survey_Belief_%"].min(),2))
print(round(climate_survey["Survey_Belief_%"].max(),2))
print(len(climate_survey["Survey_Belief_%"]))

111
2021
2022
42.85
10.52
16.0
65.25
141


In [5]:
climate_survey_rename = pd.merge(climate_survey,crosswalk[["Country Code","Meta_Name"]],left_on="country",right_on="Meta_Name",how="outer")
climate_survey_rename = climate_survey_rename.drop(columns=["country","Meta_Name"])
climate_survey_rename = climate_survey_rename.set_index(["Country Code","year"])
climate_survey_rename = climate_survey_rename.dropna(how="all")
climate_survey_rename

Survey_Worry_%  Survey_Belief_%  Survey_FutureGen_%  \
Country Code year                                                          
ARG          2021.0       88.000000        56.000000           68.000000   
             2022.0       84.594690        55.385937           70.722520   
AUS          2021.0       72.000000        50.000000           61.000000   
             2022.0       72.991429        49.768334           60.411572   
BRA          2021.0       86.000000        53.000000           74.000000   
...                             ...              ...                 ...   
ARE          2022.0       69.869396        39.027279           49.142363   
URY          2022.0       85.160842        56.147711           67.296866   
UZB          2022.0       78.558349        36.898455           36.332476   
YEM          2022.0       31.703961        20.528497           23.389677   
ZMB          2022.0       74.390219        36.073697           58.576600   

                     Survey_Personal_%  Survey_FossilMore_%  \
Country Code year                                             
ARG          2021.0          38.000000            21.000000   
             2022.0          39.090304            19.858865   
AUS          2021.0          19.000000            11.000000   
             2022.0          21.170702            12.702930   
BRA          2021.0          56.000000            27.000000   
...                                ...                  ...   
ARE          2022.0          32.829407            41.677586   
URY          2022.0          32.039047            19.608326   
UZB          2022.0          13.533452            24.500367   
YEM          2022.0          11.408358            38.808648   
ZMB          2022.0          50.957764            55.527043   

                     Survey_FossilLess_%  
Country Code year                         
ARG          2021.0            58.000000  
             2022.0            60.063782  
AUS          2021.0            71.000000  
             2022.0            66.152537  
BRA          2021.0            54.000000  
...                                  ...  
ARE          2022.0            29.960110  
URY          2022.0            61.067272  
UZB          2022.0            41.988850  
YEM          2022.0            18.752107  
ZMB          2022.0            27.255376  

[141 rows x 6 columns]

### World development index -  $\color{royalblue}{\text{political economy}}$

In [6]:
world_develpment = pd.read_csv("../_data_process/_all_temporal_world_development.csv")
world_develpment = world_develpment.set_index(["Country Code","year"])
world_develpment = world_develpment.drop(columns="WDI_EnergyGDP_kgOilEq/$1k")
world_develpment

,,GDPpc_2015$,GDPpc_2017$,Population,WDI_Manu_GDP_%,WDI_CoalRents_%,WDI_OilRents_%,WDI_NGRents_%,WDI_NG_El_%,WDI_Coal_El_%,WDI_Fossil_El_%,logGDPpc
Country Code,year,,,,,,,,,,,
AFG,1960,NaN,NaN,8622466.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SYC,1960,3218.664448,NaN,41700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SLE,1960,497.418491,NaN,2301310.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SGP,1960,3611.953651,NaN,1646400.0,10.577547,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SXM,1960,NaN,NaN,2646.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
DEU,2022,43032.142085,53560.091056,84079811.0,18.450473,NaN,NaN,NaN,16.490566,31.046312,NaN,10.888559
GRC,2022,20167.558903,31516.636854,10566531.0,8.806199,NaN,NaN,NaN,37.299770,10.411900,NaN,10.358271
HUN,2022,16288.987145,35254.503304,9683505.0,17.181391,NaN,NaN,NaN,24.943691,8.164414,NaN,10.470349


In [7]:
world_develpment_stats = world_develpment.reset_index()
print(len(world_develpment_stats["Country Code"].unique()))
print(world_develpment_stats["year"].min())
print(world_develpment_stats["year"].max())
print(round(world_develpment_stats["logGDPpc"].mean(),2))
print(round(world_develpment_stats["logGDPpc"].std(),2))
print(round(world_develpment_stats["logGDPpc"].min(),2))
print(round(world_develpment_stats["logGDPpc"].max(),2))
print(len(world_develpment_stats["logGDPpc"]))

266
1960
2022
9.13
1.17
6.08
11.97
16121


In [8]:
print(round(world_develpment_stats["WDI_Coal_El_%"].mean(),2))
print(round(world_develpment_stats["WDI_Coal_El_%"].std(),2))
print(round(world_develpment_stats["WDI_Coal_El_%"].min(),2))
print(round(world_develpment_stats["WDI_Coal_El_%"].max(),2))
print(len(world_develpment_stats["WDI_Coal_El_%"]))

18.65
25.74
0.0
100.0
16121


In [9]:
print(round(world_develpment_stats["WDI_NG_El_%"].mean(),2))
print(round(world_develpment_stats["WDI_NG_El_%"].std(),2))
print(round(world_develpment_stats["WDI_NG_El_%"].min(),2))
print(round(world_develpment_stats["WDI_NG_El_%"].max(),2))
print(len(world_develpment_stats["WDI_NG_El_%"]))

18.1
26.0
0.0
100.0
16121


### Energy statistical review -  $\color{royalblue}{\text{political economy}}$

In [10]:
energy_stats = pd.read_csv("../_data_process/_all_temporal_BPstats.csv")
energy_stats

,country,year,BP_GasReserve_tM3,BP_GasProduction_bM3,BP_CoalProduction_mTon,BP_CoalReserve_mTon,BP_GasR2P_yr,BP_CoalR2P_yr
0,Canada,1980,2.429700,71.227950,NaN,NaN,34.111609,NaN
1,Mexico,1980,1.781325,25.115242,NaN,NaN,70.926055,NaN
2,US,1980,5.396133,525.090747,NaN,NaN,10.276572,NaN
3,Total North America,1980,9.607158,621.433939,NaN,NaN,15.459661,NaN
4,Argentina,1980,0.624975,8.164624,NaN,NaN,76.546699,NaN
...,...,...,...,...,...,...,...,...
4946,Middle East,2016,NaN,NaN,NaN,NaN,NaN,500.0
4947,Middle East,2017,NaN,NaN,NaN,NaN,NaN,500.0
4948,Middle East,2018,NaN,NaN,NaN,NaN,NaN,500.0
4949,Middle East,2019,NaN,NaN,NaN,NaN,NaN,500.0


In [11]:
energy_stats_rename = pd.merge(energy_stats,crosswalk[["Country Code","BP_Name"]],left_on="country",right_on="BP_Name",how="outer")
energy_stats_rename = energy_stats_rename.drop(columns=["country","BP_Name"])
energy_stats_rename = energy_stats_rename[energy_stats_rename["year"].isna()==False]
energy_stats_rename = energy_stats_rename[energy_stats_rename["Country Code"].isna()==False]
energy_stats_rename = energy_stats_rename.drop_duplicates()
energy_stats_rename = energy_stats_rename.set_index(["Country Code","year"])
energy_stats_rename["LogBP_GasR2P_yr"] = np.log(energy_stats_rename["BP_GasR2P_yr"])
energy_stats_rename["LogBP_GasR2P_yr"] = energy_stats_rename["LogBP_GasR2P_yr"].fillna(0)
energy_stats_rename

C:\Users\jx920\AppData\Roaming\Python\Python38\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


BP_GasReserve_tM3  BP_GasProduction_bM3  \
Country Code year                                              
CAN          1980.0           2.429700             71.227950   
             1981.0           2.498925             68.827650   
             1982.0           2.526225             72.247125   
             1983.0           2.547675             67.951350   
             1984.0           2.738775             74.475975   
...                                ...                   ...   
ZMB          2017.0                NaN                   NaN   
             2018.0                NaN                   NaN   
             2019.0                NaN                   NaN   
             2020.0                NaN                   NaN   
             2021.0                NaN                   NaN   

                     BP_CoalProduction_mTon  BP_CoalReserve_mTon  \
Country Code year                                                  
CAN          1980.0                     NaN                  NaN   
             1981.0                  40.088                  NaN   
             1982.0                  42.907                  NaN   
             1983.0                  44.807                  NaN   
             1984.0                  57.402                  NaN   
...                                     ...                  ...   
ZMB          2017.0                     NaN                 10.0   
             2018.0                     NaN                 10.0   
             2019.0                     NaN                 10.0   
             2020.0                     NaN                 10.0   
             2021.0                     NaN                 10.0   

                     BP_GasR2P_yr  BP_CoalR2P_yr  LogBP_GasR2P_yr  
Country Code year                                                  
CAN          1980.0     34.111609            NaN         3.529638  
             1981.0     36.306993            NaN         3.592010  
             1982.0     34.966443            NaN         3.554389  
             1983.0     37.492633            NaN         3.624144  
             1984.0     36.773940            NaN         3.604789  
...                           ...            ...              ...  
ZMB          2017.0           NaN            NaN         0.000000  
             2018.0           NaN            NaN         0.000000  
             2019.0           NaN            NaN         0.000000  
             2020.0           NaN            NaN         0.000000  
             2021.0           NaN            NaN         0.000000  

[2813 rows x 7 columns]

### Fuel trade diversity and gravity -  $\color{royalblue}{\text{political economy}}$
- so far only includes LNG data

In [12]:
gas_trade = pd.read_csv("../_data_process/_temporal_BACI_DGD_Gas_both.csv")
gas_trade = gas_trade.rename(columns={"country":"Country Code"})
gas_trade = gas_trade[gas_trade["year"]!=0]
gas_trade = gas_trade.set_index(["Country Code","year"])
#gas_trade["LogNetValue_Gas_Import"] = np.log(gas_trade["NetValue_Gas_Import"])
gas_trade

HHI_Gas_Import  contiguity_Gas_Import  agree_Gas_Import  \
Country Code year                                                              
BEL          1995.0        0.296619               0.496219          1.000000   
             1996.0        0.316463               0.460321          1.000000   
             1997.0        0.305224               0.402619          1.000000   
             1998.0        0.340924               0.195566          1.000000   
             1999.0        0.387019               0.093431          0.999987   
...                             ...                    ...               ...   
VCT          2018.0        0.000000               0.000000          0.000000   
             2019.0        0.000000               0.000000          0.000000   
             2020.0        0.000000               0.000000          0.000000   
CYM          2018.0        0.000000               0.000000          0.000000   
GNB          2021.0        0.000000               0.000000          0.000000   

                     sanction_Gas_Import  common_language_Gas_Import  \
Country Code year                                                      
BEL          1995.0             0.000000                    0.760876   
             1996.0             0.000000                    0.694677   
             1997.0             0.000000                    0.707491   
             1998.0             0.000000                    0.568168   
             1999.0             0.000013                    0.445870   
...                                  ...                         ...   
VCT          2018.0             0.000000                    0.000000   
             2019.0             0.000000                    0.000000   
             2020.0             0.000000                    0.000000   
CYM          2018.0             0.000000                    0.000000   
GNB          2021.0             0.000000                    0.000000   

                     colony_Gas_Import  HHI_Gas_Export  contiguity_Gas_Export  \
Country Code year                                                               
BEL          1995.0           0.416791        0.490196               0.906559   
             1996.0           0.408114        0.659054               0.908745   
             1997.0           0.352952        0.981658               0.008484   
             1998.0           0.136150        0.897484               0.946805   
             1999.0           0.047905        0.997479               0.000082   
...                                ...             ...                    ...   
VCT          2018.0           0.000000        1.000000               0.000000   
             2019.0           0.000000        1.000000               0.000000   
             2020.0           0.000000        1.000000               0.000000   
CYM          2018.0           0.000000        1.000000               0.000000   
GNB          2021.0           0.000000        1.000000               0.000000   

                     agree_Gas_Export  sanction_Gas_Export  \
Country Code year                                            
BEL          1995.0          0.998680             0.000000   
             1996.0          1.000000             0.000000   
             1997.0          1.000000             0.990757   
             1998.0          1.000000             0.000000   
             1999.0          0.998835             0.998738   
...                               ...                  ...   
VCT          2018.0          1.000000             0.000000   
             2019.0          1.000000             0.000000   
             2020.0          0.000000             0.000000   
CYM          2018.0          0.000000             0.000000   
GNB          2021.0          1.000000             0.000000   

                     common_language_Gas_Export  colony_Gas_Export  \
Country Code year                                                    
BEL          1995.0                    0.9

In [13]:
coal_trade = pd.read_csv("../_data_process/_temporal_BACI_DGD_Coal_both.csv")
coal_trade = coal_trade.rename(columns={"country":"Country Code"})
coal_trade = coal_trade[coal_trade["year"]!=0]
coal_trade = coal_trade.set_index(["Country Code","year"])
#coal_trade["LogNetValue_Coal_Import"] = np.log(coal_trade["NetValue_Coal_Import"])
#coal_trade["LogNetValue_Coal_Import"] = coal_trade["LogNetValue_Coal_Import"].fillna(0)
coal_trade

HHI_Coal_Import  contiguity_Coal_Import  \
Country Code year                                              
USA          1995.0         0.292268                0.188110   
             1996.0         0.350755                0.188541   
             1997.0         0.342787                0.160638   
             1998.0         0.298241                0.147175   
             1999.0         0.394687                0.113625   
...                              ...                     ...   
GIN          2020.0         0.000000                0.000000   
SLB          2013.0         0.000000                0.000000   
SPM          2015.0         0.000000                0.000000   
TCD          2015.0         0.000000                0.000000   
SSD          2017.0         0.000000                0.000000   

                     agree_Coal_Import  sanction_Coal_Import  \
Country Code year                                              
USA          1995.0           0.759553              0.188122   
             1996.0           0.885144              0.188541   
             1997.0           0.909158              0.160922   
             1998.0           0.802874              0.458828   
             1999.0           0.867115              0.009602   
...                                ...                   ...   
GIN          2020.0           0.000000              0.000000   
SLB          2013.0           0.000000              0.000000   
SPM          2015.0           0.000000              0.000000   
TCD          2015.0           0.000000              0.000000   
SSD          2017.0           0.000000              0.000000   

                     common_language_Coal_Import  colony_Coal_Import  \
Country Code year                                                      
USA          1995.0                     0.999630            0.000013   
             1996.0                     0.999242            0.000058   
             1997.0                     0.994025            0.001955   
             1998.0                     0.996489            0.004816   
             1999.0                     0.997151            0.007328   
...                                          ...                 ...   
GIN          2020.0                     0.000000            0.000000   
SLB          2013.0                     0.000000            0.000000   
SPM          2015.0                     0.000000            0.000000   
TCD          2015.0                     0.000000            0.000000   
SSD          2017.0                     0.000000            0.000000   

                     HHI_Coal_Export  contiguity_Coal_Export  \
Country Code year                                              
USA          1995.0         0.079788                0.131941   
             1996.0         0.073304                0.099616   
             1997.0         0.080748                0.129748   
             1998.0         0.096629                0.200652   
             1999.0         0.137138                0.270839   
...                              ...                     ...   
GIN          2020.0         1.000000                0.000000   
SLB          2013.0         1.000000                0.000000   
SPM          2015.0         1.000000                0.000000   
TCD          2015.0         1.000000                0.000000   
SSD          2017.0         1.000000                0.000000   

                     agree_Coal_Export  sanction_Coal_Export  \
Country Code year                                              
USA          1995.0           0.131942              0.700034   
             1996.0           0.099616              0.676095   
             1997.0           0.157845              0.706367   
             1998.0           0.231414              0.675392   
             1999.0           0.291520              0.874754   
...                                ...                   ...   
GIN          2020.0           1.000000              0.000000   
SLB      

In [14]:
gas_trade_stats = gas_trade.reset_index()
print(len(gas_trade_stats["Country Code"].unique()))
print(gas_trade_stats["year"].min())
print(gas_trade_stats["year"].max())

223
1995.0
2021.0


In [15]:
"""
print(round(gas_trade_stats["HHI_Gas_Export"].mean(),2))
print(round(LNG_trade_stats["HHI_LNG_Export"].std(),2))
print(round(LNG_trade_stats["HHI_LNG_Export"].min(),2))
print(round(LNG_trade_stats["HHI_LNG_Export"].max(),2))
print(len(LNG_trade_stats["HHI_LNG_Export"]))
"""

'\nprint(round(gas_trade_stats["HHI_Gas_Export"].mean(),2))\nprint(round(LNG_trade_stats["HHI_LNG_Export"].std(),2))\nprint(round(LNG_trade_stats["HHI_LNG_Export"].min(),2))\nprint(round(LNG_trade_stats["HHI_LNG_Export"].max(),2))\nprint(len(LNG_trade_stats["HHI_LNG_Export"]))\n'

In [16]:
"""
print(round(LNG_trade_stats["HHI_LNG_Import"].mean(),2))
print(round(LNG_trade_stats["HHI_LNG_Import"].std(),2))
print(round(LNG_trade_stats["HHI_LNG_Import"].min(),2))
print(round(LNG_trade_stats["HHI_LNG_Import"].max(),2))
print(len(LNG_trade_stats["HHI_LNG_Import"]))
"""

'\nprint(round(LNG_trade_stats["HHI_LNG_Import"].mean(),2))\nprint(round(LNG_trade_stats["HHI_LNG_Import"].std(),2))\nprint(round(LNG_trade_stats["HHI_LNG_Import"].min(),2))\nprint(round(LNG_trade_stats["HHI_LNG_Import"].max(),2))\nprint(len(LNG_trade_stats["HHI_LNG_Import"]))\n'

### POLCON dataset -  $\color{darkseagreen}{\text{institutional}}$

In [17]:
POLCON = pd.read_csv("../_data_process/_all_temporal_POLCON.csv")
POLCON = POLCON.rename(columns={"ctrynm":"Country Code"}).drop(columns="country_name")
POLCON = POLCON.set_index(["Country Code","year"])
POLCON

l1   l2   j   f  aligne_l1  aligne_l2  alignl1_l2  \
Country Code year                                                       
USA          1920  1.0  1.0 NaN NaN        0.0        0.0         1.0   
             1921  1.0  1.0 NaN NaN        1.0        1.0         1.0   
             1922  1.0  1.0 NaN NaN        1.0        1.0         1.0   
             1923  1.0  1.0 NaN NaN        1.0        1.0         1.0   
             1924  1.0  1.0 NaN NaN        1.0        1.0         1.0   
...                ...  ...  ..  ..        ...        ...         ...   
WSM          2017  NaN  NaN NaN NaN        1.0        0.0         0.0   
             2018  NaN  NaN NaN NaN        1.0        0.0         0.0   
             2019  NaN  NaN NaN NaN        1.0        0.0         0.0   
             2020  NaN  NaN NaN NaN        1.0        0.0         0.0   
             2021  NaN  NaN NaN NaN        1.0        0.0         0.0   

                   aligne_j  
Country Code year            
USA          1920       NaN  
             1921       NaN  
             1922       NaN  
             1923       NaN  
             1924       NaN  
...                     ...  
WSM          2017       NaN  
             2018       NaN  
             2019       NaN  
             2020       NaN  
             2021       NaN  

[13597 rows x 8 columns]

### Power sector reform tracker -  $\color{darkseagreen}{\text{institutional}}$
- dropping repeat data for Venezuela, Macedonia, and Kyrgystan (using the most recent); no matching set for Central African Republic

In [18]:
PSRT = pd.read_csv("../_data_process/_all_temporal_power_reform.csv")
PSRT

,country,year,source,R_IndepProducer,R_Private,R_Unbundle,R_WholeSale,R_IndepReg,R_Choice,R_Liberalization,R_Corp
0,Greece,1999.0,Erdogdu 2011,0,0,1,0,1,0,1,0
1,Greece,2000.0,Erdogdu 2011,0,0,1,0,1,0,1,1
2,Greece,2001.0,Erdogdu 2011,0,0,1,0,1,1,1,1
3,Greece,2002.0,Erdogdu 2011,0,1,1,0,1,1,1,1
4,Greece,2003.0,Erdogdu 2011,0,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
7293,Mozambique,2018.0,assumption,1,0,0,0,1,0,1,1
7294,Mozambique,2019.0,assumption,1,0,0,0,1,0,1,1
7295,Mozambique,2020.0,assumption,1,0,0,0,1,0,1,1
7296,Mozambique,2021.0,assumption,1,0,0,0,1,0,1,1


In [19]:
print(len(PSRT["country"].unique()))
print(PSRT["year"].min())
print(PSRT["year"].max())

178
1982.0
2022.0


In [20]:
print(round(PSRT["R_Choice"].mean(),2))
print(round(PSRT["R_Choice"].std(),2))
print(round(PSRT["R_Choice"].min(),2))
print(round(PSRT["R_Choice"].max(),2))
print(len(PSRT["R_Choice"]))

0.13
0.34
0
1
7298


In [21]:
PSRT_rename = pd.merge(PSRT,crosswalk[["Country Code","Reform_Name"]],left_on="country",right_on="Reform_Name",how="outer")
PSRT_rename = PSRT_rename.drop(columns=["country","Reform_Name","source"])
PSRT_rename = PSRT_rename.set_index(["Country Code","year"])
PSRT_rename = PSRT_rename.dropna(how="all")
PSRT_rename

R_IndepProducer  R_Private  R_Unbundle  R_WholeSale  \
Country Code year                                                          
GRC          1999.0              0.0        0.0         1.0          0.0   
             2000.0              0.0        0.0         1.0          0.0   
             2001.0              0.0        0.0         1.0          0.0   
             2002.0              0.0        1.0         1.0          0.0   
             2003.0              0.0        1.0         1.0          0.0   
...                              ...        ...         ...          ...   
MOZ          2018.0              1.0        0.0         0.0          0.0   
             2019.0              1.0        0.0         0.0          0.0   
             2020.0              1.0        0.0         0.0          0.0   
             2021.0              1.0        0.0         0.0          0.0   
             2022.0              1.0        0.0         0.0          0.0   

                     R_IndepReg  R_Choice  R_Liberalization  R_Corp  
Country Code year                                                    
GRC          1999.0         1.0       0.0               1.0     0.0  
             2000.0         1.0       0.0               1.0     1.0  
             2001.0         1.0       1.0               1.0     1.0  
             2002.0         1.0       1.0               1.0     1.0  
             2003.0         1.0       1.0               1.0     1.0  
...                         ...       ...               ...     ...  
MOZ          2018.0         1.0       0.0               1.0     1.0  
             2019.0         1.0       0.0               1.0     1.0  
             2020.0         1.0       0.0               1.0     1.0  
             2021.0         1.0       0.0               1.0     1.0  
             2022.0         1.0       0.0               1.0     1.0  

[7298 rows x 8 columns]

### State capacity dataset -  $\color{darkseagreen}{\text{institutional}}$

In [22]:
state_capacity = pd.read_csv("../_data_process/_all_temporal_state_capacity.csv")
state_capacity

,cntrynum,country,year,iso3,StateCapacity,StateCapacity_sd,tax_inc_tax,tax_trade_tax,taxrev_gdp
0,1.0,Afghanistan,1960.0,AFG,-1.3830,0.419495,NaN,NaN,NaN
1,1.0,Afghanistan,1961.0,AFG,-1.3620,0.411242,NaN,NaN,NaN
2,1.0,Afghanistan,1962.0,AFG,-1.3690,0.411928,NaN,NaN,NaN
3,1.0,Afghanistan,1963.0,AFG,-1.3600,0.422916,NaN,NaN,NaN
4,1.0,Afghanistan,1964.0,AFG,-1.3790,0.428595,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
8832,199.0,South Sudan,2011.0,SSD,-0.8025,0.440147,NaN,NaN,0.009247
8833,199.0,South Sudan,2012.0,SSD,-0.9320,0.403032,NaN,NaN,0.025273
8834,199.0,South Sudan,2013.0,SSD,-1.4000,0.345209,NaN,NaN,0.018151
8835,199.0,South Sudan,2014.0,SSD,-1.4010,0.335133,NaN,NaN,0.026940


In [23]:
state_capacity_rename = pd.merge(state_capacity,crosswalk[["Country Code","Hanson_Name"]],left_on="country",right_on="Hanson_Name",how="outer")
state_capacity_rename = state_capacity_rename.drop(columns=["country","Hanson_Name","cntrynum","iso3"])
state_capacity_rename = state_capacity_rename.set_index(["Country Code","year"])
state_capacity_rename = state_capacity_rename.dropna(how="all")
state_capacity_rename

StateCapacity  StateCapacity_sd  tax_inc_tax  \
Country Code year                                                   
AFG          1960.0        -1.3830          0.419495          NaN   
             1961.0        -1.3620          0.411242          NaN   
             1962.0        -1.3690          0.411928          NaN   
             1963.0        -1.3600          0.422916          NaN   
             1964.0        -1.3790          0.428595          NaN   
...                            ...               ...          ...   
SSD          2011.0        -0.8025          0.440147          NaN   
             2012.0        -0.9320          0.403032          NaN   
             2013.0        -1.4000          0.345209          NaN   
             2014.0        -1.4010          0.335133          NaN   
             2015.0        -1.3680          0.329144          NaN   

                     tax_trade_tax  taxrev_gdp  
Country Code year                               
AFG          1960.0            NaN         NaN  
             1961.0            NaN         NaN  
             1962.0            NaN         NaN  
             1963.0            NaN         NaN  
             1964.0            NaN         NaN  
...                            ...         ...  
SSD          2011.0            NaN    0.009247  
             2012.0            NaN    0.025273  
             2013.0            NaN    0.018151  
             2014.0            NaN    0.026940  
             2015.0            NaN    0.082958  

[8746 rows x 5 columns]

### Worldwide governance -  $\color{darkseagreen}{\text{institutional}}$

In [24]:
world_governance = pd.read_csv("../_data_process/_all_temporal_worldwide_governance.csv")
world_governance = world_governance.set_index(["Country Code","year"])
world_governance

WG_GovEff    WG_Reg  WG_Voice
Country Code year                                 
ABW          1996.0        NaN       NaN       NaN
             1997.0        NaN       NaN       NaN
             1998.0        NaN       NaN       NaN
             1999.0        NaN       NaN       NaN
             2000.0        NaN       NaN       NaN
...                        ...       ...       ...
ZWE          2017.0  -1.238663 -1.564137 -1.195904
             2018.0  -1.261774 -1.509489 -1.140975
             2019.0  -1.267288 -1.468613 -1.164705
             2020.0  -1.299779 -1.416232 -1.113716
             2021.0  -1.242930 -1.372497 -1.136934

[5564 rows x 3 columns]

### Policy (Fankhauser)

In [25]:
policy_fankhauser = pd.read_csv("../_data_process/_all_temporal_policy.csv")
policy_fankhauser = policy_fankhauser.rename(columns={"ISO":"Country Code","Year":"year"})
policy_fankhauser = policy_fankhauser.set_index(["Country Code","year"])
policy_fankhauser

num_supply_policy  binary_airqual  ghg_ambition_n  \
Country Code year                                                        
DZA          1980.0                0.0             0.0             0.0   
             1981.0                0.0             0.0             0.0   
             1982.0                0.0             0.0             0.0   
             1983.0                0.0             0.0             0.0   
             1984.0                0.0             0.0             0.0   
...                                ...             ...             ...   
ALB          2019.0                3.0             1.0             1.0   
             2020.0                3.0             1.0             1.0   
             2021.0                3.0             1.0             1.0   
             2022.0                3.0             1.0             1.0   
             2023.0                3.0             1.0             1.0   

                     cw_2030  annex_one  annex_two  
Country Code year                                   
DZA          1980.0      0.0        0.0        0.0  
             1981.0      0.0        0.0        0.0  
             1982.0      0.0        0.0        0.0  
             1983.0      0.0        0.0        0.0  
             1984.0      0.0        0.0        0.0  
...                      ...        ...        ...  
ALB          2019.0      1.0        0.0        0.0  
             2020.0      1.0        0.0        0.0  
             2021.0      1.0        0.0        0.0  
             2022.0      1.0        0.0        0.0  
             2023.0      1.0        0.0        0.0  

[8272 rows x 6 columns]

## Compile datasets together

In [26]:
data_list = [climate_survey_rename, world_develpment, energy_stats_rename,
             gas_trade, coal_trade,POLCON, PSRT_rename, state_capacity_rename, world_governance,policy_fankhauser]
for count in np.arange(len(data_list)):
    data = data_list[count]
    if count==0:
        data_compile = data
    else:
        data_compile = pd.merge(data_compile,data,on=["Country Code","year"],how="outer")
data_compile

Survey_Worry_%  Survey_Belief_%  Survey_FutureGen_%  \
Country Code year                                                          
ARG          2021.0       88.000000        56.000000           68.000000   
             2022.0       84.594690        55.385937           70.722520   
AUS          2021.0       72.000000        50.000000           61.000000   
             2022.0       72.991429        49.768334           60.411572   
BRA          2021.0       86.000000        53.000000           74.000000   
...                             ...              ...                 ...   
COK          1994.0             NaN              NaN                 NaN   
             2022.0             NaN              NaN                 NaN   
             2023.0             NaN              NaN                 NaN   
LBY          2023.0             NaN              NaN                 NaN   
ALB          2023.0             NaN              NaN                 NaN   

                     Survey_Personal_%  Survey_FossilMore_%  \
Country Code year                                             
ARG          2021.0          38.000000            21.000000   
             2022.0          39.090304            19.858865   
AUS          2021.0          19.000000            11.000000   
             2022.0          21.170702            12.702930   
BRA          2021.0          56.000000            27.000000   
...                                ...                  ...   
COK          1994.0                NaN                  NaN   
             2022.0                NaN                  NaN   
             2023.0                NaN                  NaN   
LBY          2023.0                NaN                  NaN   
ALB          2023.0                NaN                  NaN   

                     Survey_FossilLess_%   GDPpc_2015$   GDPpc_2017$  \
Country Code year                                                      
ARG          2021.0            58.000000  12402.490791  21527.196137   
             2022.0            60.063782           NaN           NaN   
AUS          2021.0            71.000000  59339.684345  49774.340702   
             2022.0            66.152537           NaN           NaN   
BRA          2021.0            54.000000   8621.730590  14735.581783   
...                                  ...           ...           ...   
COK          1994.0                  NaN           NaN           NaN   
             2022.0                  NaN           NaN           NaN   
             2023.0                  NaN           NaN           NaN   
LBY          2023.0                  NaN           NaN           NaN   
ALB          2023.0                  NaN           NaN           NaN   

                      Population  WDI_Manu_GDP_%  ...  taxrev_gdp  WG_GovEff  \
Country Code year                                 ...                          
ARG          2021.0   45808747.0       15.479166  ...         NaN  -0.356662   
             2022.0          NaN             NaN  ...         NaN        NaN   
AUS          2021.0   25688079.0        5.548834  ...         NaN   1.512928   
             2022.0          NaN             NaN  ...         NaN        NaN   
BRA          2021.0  214326223.0       10.223030  ...         NaN  -0.460292   
...                          ...             ...  ...         ...        ...   
COK          1994.0          NaN             NaN  ...         NaN        NaN   
             2022.0          NaN             NaN  ...         NaN        NaN   
             2023.0          NaN             NaN  ...         NaN        NaN   
LBY          2023.0          NaN             NaN  ...         NaN        NaN   
ALB          2023.0          NaN             NaN  ...         NaN        NaN   

                       WG_Reg  WG_Voice  num_supply_policy  binary_airqual  \
Country Code year                                                            
ARG          2021.0 -0.618145  0.619620               12.0             1.0   


In [27]:
# choose only the countries in the crosswalk file
data_compile_format = data_compile.reset_index()
cw_countries = crosswalk["Country Code"].unique()
data_compile_format_crop = data_compile_format[data_compile_format["Country Code"].isin(cw_countries)]
data_compile_format_crop = data_compile_format_crop.sort_values(by="year")
data_compile_format_crop

,Country Code,year,Survey_Worry_%,Survey_Belief_%,Survey_FutureGen_%,Survey_Personal_%,Survey_FossilMore_%,Survey_FossilLess_%,GDPpc_2015$,GDPpc_2017$,...,taxrev_gdp,WG_GovEff,WG_Reg,WG_Voice,num_supply_policy,binary_airqual,ghg_ambition_n,cw_2030,annex_one,annex_two
17716,ARG,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17676,CHL,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17534,BOL,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17494,BRA,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17454,PER,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26000,SEN,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,1.0,0.0,0.0,0.0,0.0
26001,SGP,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,1.0,0.0,0.0,0.0,0.0
26002,SVK,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,0.0,0.0,0.0,1.0,0.0
25992,PAK,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,1.0,1.0,1.0,0.0,0.0


In [28]:
# fill the production and reserve data as 0 if they do not appear in the original list of countries
unique_ff_producers = energy_stats_rename.reset_index()["Country Code"].unique()
not_producers = data_compile_format_crop[data_compile_format_crop["Country Code"].isin(unique_ff_producers)==False].index
data_compile_format_crop.loc[not_producers,energy_stats_rename.columns] = data_compile_format_crop.loc[not_producers,energy_stats_rename.columns].fillna(0)
data_compile_format_crop = data_compile_format_crop.sort_values(by="year")
data_compile_format_crop

,Country Code,year,Survey_Worry_%,Survey_Belief_%,Survey_FutureGen_%,Survey_Personal_%,Survey_FossilMore_%,Survey_FossilLess_%,GDPpc_2015$,GDPpc_2017$,...,taxrev_gdp,WG_GovEff,WG_Reg,WG_Voice,num_supply_policy,binary_airqual,ghg_ambition_n,cw_2030,annex_one,annex_two
17716,ARG,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21247,ETH,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20570,LBR,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19904,NOR,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19762,FIN,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26082,TGO,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,0.0,1.0,1.0,0.0,0.0
26083,BRN,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0
26084,BDI,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,0.0,0.0,1.0,0.0,0.0
26105,XKX,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# select the countries and unique year in the LNG trade dataset
# within those years, the countries not in the dataset are set to 0
unique_trade = gas_trade.reset_index()["Country Code"].unique()
unique_trade_years = gas_trade.reset_index()["year"].unique()
not_trade_table = data_compile_format_crop[data_compile_format_crop["Country Code"].isin(unique_trade)==False]
not_trade_table_years = not_trade_table[not_trade_table["year"].isin(unique_trade_years)]
not_trade = not_trade_table_years.index
data_compile_format_crop.loc[not_trade,gas_trade.columns] = data_compile_format_crop.loc[not_trade,gas_trade.columns].fillna(0)
data_compile_format_crop = data_compile_format_crop.sort_values(by="year")
data_compile_format_crop[["num_supply_policy","binary_airqual","ghg_ambition_n","cw_2030","annex_one","annex_two"]] = data_compile_format_crop[["num_supply_policy","binary_airqual","ghg_ambition_n","cw_2030","annex_one","annex_two"]].fillna(0)
data_compile_format_crop

,Country Code,year,Survey_Worry_%,Survey_Belief_%,Survey_FutureGen_%,Survey_Personal_%,Survey_FossilMore_%,Survey_FossilLess_%,GDPpc_2015$,GDPpc_2017$,...,taxrev_gdp,WG_GovEff,WG_Reg,WG_Voice,num_supply_policy,binary_airqual,ghg_ambition_n,cw_2030,annex_one,annex_two
17716,ARG,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
17534,BOL,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
17494,BRA,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
17454,PER,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
17414,ECU,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26002,SVK,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,0.0,0.0,0.0,1.0,0.0
25962,GUY,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,1.0,0.0,0.0,0.0,0.0
25983,MNG,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8.0,1.0,1.0,0.0,0.0,0.0
25995,POL,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,1.0,0.0,0.0,1.0,0.0


In [30]:
data_compile_export = data_compile_format_crop.set_index(["Country Code","year"])
data_compile_export.to_csv("_all_temporal_indep_var.csv")